In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing import sequence

# Load the IMDB dataset
max_features = 10000  # Only consider the top 10,000 words in the dataset
max_len = 500  # Cut reviews after 500 words
batch_size = 32

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

# Build the LSTM model
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

# Train the model
history = model.fit(x_train, y_train, epochs=5, batch_size=batch_size, validation_split=0.2)

# Evaluate the model on the test set
results = model.evaluate(x_test, y_test)
print(f'Test loss: {results[0]}, Test accuracy: {results[1]}')


17464789/17464789 [==============================] - 1s 0us/step
Epoch 1/5
625/625 [==============================] - 120s 188ms/step - loss: 0.4709 - acc: 0.7732 - val_loss: 0.3559 - val_acc: 0.8458
Epoch 2/5
625/625 [==============================] - 118s 189ms/step - loss: 0.2829 - acc: 0.8885 - val_loss: 0.2894 - val_acc: 0.8856
Epoch 3/5
625/625 [==============================] - 125s 200ms/step - loss: 0.2358 - acc: 0.9087 - val_loss: 0.3390 - val_acc: 0.8798
Epoch 4/5
625/625 [==============================] - 118s 188ms/step - loss: 0.2082 - acc: 0.9232 - val_loss: 0.2962 - val_acc: 0.8846
Epoch 5/5
782/782 [==============================] - 42s 53ms/step - loss: 0.2995 - acc: 0.8781
Test loss: 0.29946014285087585, Test accuracy: 0.8780800104141235


In [2]:
import numpy as np

class LSTMCell:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Weights and biases
        self.Wf = np.random.randn(hidden_size, input_size + hidden_size)
        self.Wi = np.random.randn(hidden_size, input_size + hidden_size)
        self.Wc = np.random.randn(hidden_size, input_size + hidden_size)
        self.Wo = np.random.randn(hidden_size, input_size + hidden_size)

        self.bf = np.zeros((hidden_size, 1))
        self.bi = np.zeros((hidden_size, 1))
        self.bc = np.zeros((hidden_size, 1))
        self.bo = np.zeros((hidden_size, 1))

        # Cell state and hidden state
        self.c = np.zeros((hidden_size, 1))
        self.h = np.zeros((hidden_size, 1))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def forward(self, x):
        # Concatenate input and hidden state
        concat_input = np.vstack((self.h, x))

        # Forget gate
        f = self.sigmoid(np.dot(self.Wf, concat_input) + self.bf)

        # Input gate
        i = self.sigmoid(np.dot(self.Wi, concat_input) + self.bi)

        # Cell state candidates
        c_hat = self.tanh(np.dot(self.Wc, concat_input) + self.bc)

        # Update cell state
        self.c = f * self.c + i * c_hat

        # Output gate
        o = self.sigmoid(np.dot(self.Wo, concat_input) + self.bo)

        # Update hidden state
        self.h = o * self.tanh(self.c)

        return self.h, self.c

# Example usage
input_size = 3
hidden_size = 2

lstm_cell = LSTMCell(input_size, hidden_size)

# Input sequence
sequence = [np.array([[1], [2], [3]]) for _ in range(5)]

for step in sequence:
    output, cell_state = lstm_cell.forward(step)
    print("Output:", output)


Output: [[0.12130812]
 [0.72008675]]
Output: [[0.19589049]
 [0.86564405]]
Output: [[0.2622228 ]
 [0.89752156]]
Output: [[0.32565316]
 [0.90771471]]
Output: [[0.38668682]
 [0.91141334]]
